In [1]:
import pandas as pd
import datetime as dt
from logger_settings import logger
import phase as ph

pd.options.display.max_colwidth = 10
pd.options.display.max_rows = None

ticks_columns = {
    'last_price': float,
    'last_traded_quantity': int,
    'total_buy_quantity': int,
    'total_sell_quantity': int,
    'last_trade_time': 'datetime64[ns]',
    'volume_traded': 'int',
    'oi': int,
}
tdf = pd.read_json('ticks-formatted.json', dtype=ticks_columns, convert_dates={'last_trade_time': '%Y-%m-%d %H:%M:%S'})
columns_to_drop = set(tdf.columns).difference(ticks_columns.keys())
tdf.drop(columns=columns_to_drop, inplace=True)
tdf.set_index('last_trade_time', inplace=True, drop=False)
tdf['volume'] = tdf.volume_traded - tdf.volume_traded.shift(1)
tdf.drop('volume_traded', inplace=True, axis=1)
tdf.fillna({'volume': 0}, inplace=True)

class Settings:
    # MIN_UP_PC_CHG = 0.5 / 100 # .5%
    # MIN_DOWN_PC_CHG = -0.5 / 100 # -.5%
    # BREACH_PC = 5 / 100 # 5% Percentage of continuation phase gains required to breach the trend
    # LAST_TICKS_BREACH = {ph.Direction.UP: 5, ph.Direction.DOWN: 5}
    QUANTITY = 50
    CONFIRM_TICKS = 5 # 5 Seconds
    # TP = .01 # 2%

In [2]:
class ContinuationPhase(ph.Phase):
    def __init__(self, ticks, pm):
        self.phase_type = ph.Phase.TYPE_CONT
        self.t_start = ticks[0]
        self.t_end = ticks[-1]
        self.status = self.STATUS_INITIATED
        # self.min_pc = None
        self.direction = None
        self.pm = pm
        self.last_processed_tid = ticks[-1].id
        self.started_at = None
        self.terminated_at = None
        self.hard_retraced_at = None
        self.soft_retraced_at = None
        self.second_last_5sec = ph.Candle(ticks, confirm_ticks=self.pm.settings.CONFIRM_TICKS)
        self.last_5sec = ph.Candle([], confirm_ticks=self.pm.settings.CONFIRM_TICKS)
        self.up_sum, self.down_sum = self.get_sums(ticks)
        self.confidence_at_confirmation = None
        self.svac = 0 # Second last candle volume at confirmation
        self.lvac = 0 # last candle volume at confirmation

    def get_sums(self, ticks):
        if len(ticks) == 1:
            return 0, 0
        cur_price = ticks[0].last_price
        up_sum = down_sum = 0
        for i in range(1, len(ticks)):
            ch = ticks[i].last_price - cur_price
            if ch >= 0:
                up_sum += ch
            else:
                down_sum += ch
            cur_price = ticks[i].last_price
        return up_sum, down_sum

    @property
    def confidence(self):
        total = (self.up_sum + abs(self.down_sum))
        if total == 0:
            logger.info(f"u: {self.up_sum}, d: {self.down_sum}")
            return 0
        if self.direction == ph.Direction.UP:
            return self.up_sum * 100 / total
        return self.down_sum * 100 / total

    def start_phase(self):
        if len(self.last_5sec) < 1:
            return False
        is_red_breach = self.second_last_5sec.IS_RED and self.last_5sec.CLOSE >= self.second_last_5sec.OPEN
        is_green_breach = (not self.second_last_5sec.IS_RED) and self.last_5sec.CLOSE <= self.second_last_5sec.OPEN
        if is_red_breach or is_green_breach:
            raise ph.PhaseStartFailed(f'failed to start phase at: {self.last_5sec.end_id}')
        # if self.second_last_5sec.volume > self.last_5sec.volume:
        #     return False
        price_change = self.last_5sec.CLOSE - self.second_last_5sec.OPEN
        self.direction = ph.Direction.UP if price_change >= 0 else ph.Direction.DOWN
        # self.min_pc = Settings.MIN_UP_PC_CHG if price_change >= 0 else Settings.MIN_DOWN_PC_CHG
        self.status = self.STATUS_STARTED
        self.started_at = self.last_5sec[-1]
        self.confidence_at_confirmation = self.confidence
        self.svac = self.second_last_5sec.volume
        self.lvac = self.last_5sec.volume
        return True

    def update_sum(self):
        if self.t_start.id == 51463:
            print(f"before u: {self.up_sum}, d: {self.down_sum}")
        ch = self.pm.ticks[-1].last_price - self.pm.ticks[-2].last_price
        if ch >= 0:
            self.up_sum += ch
        else:
            self.down_sum += ch
        if self.t_start.id == 51463:
            print(f"after u: {self.up_sum}, d: {self.down_sum}, ch: {ch}, -1 id: {self.pm.ticks[-1].id}, -2 id: {self.pm.ticks[-2].id}")

    def process(self, tick):
        """
        if it breaks previous open or previous nth tick
        """
        self.last_processed_tid = tick.id
        self.update_last_nsec(tick)
        self.update_sum()
        self.t_end = tick
        if self.status == self.STATUS_INITIATED:
            has_started = self.start_phase()
            if not has_started:
                return
            self.pm.on_phase_start(self, tick)
            return
        elif self.status == self.STATUS_TERM or self.status == self.STATUS_HARD_RETR:
            return
        change = self.last_5sec.CLOSE - self.second_last_5sec.OPEN
        open_change = self.last_5sec.CLOSE - self.t_start.last_price
        last_5sec_breach = (self.direction == ph.Direction.UP and change < 0) or (self.direction == ph.Direction.DOWN and change > 0)
        open_breach = (self.direction == ph.Direction.UP and open_change < 0) or (self.direction == ph.Direction.DOWN and open_change > 0)
        if last_5sec_breach or open_breach:
            self.status = self.STATUS_HARD_RETR
            self.hard_retraced_at = tick
            raise ph.PhaseHardRetraceException(f"Continuation Phase {self} hard retracing at: {tick}")
        self.status = self.STATUS_ONG

In [3]:
class MyPhaseManager(ph.PhaseManager):
    def __init__(self, instrument, settings):
        super().__init__(instrument, settings)

    def process(self, tick):
        if self.current_phase is None:
            # First call
            self.current_phase = ContinuationPhase(ticks=[tick], pm=self)
            return
        # Has existing phase
        try:
            self.current_phase.process(tick)
            self.on_ongoing(self.current_phase, tick)
        except ph.PhaseSoftRetraceException:
            self.on_soft_retracel(self.current_phase, tick)
        except ph.PhaseHardRetraceException:
            self.on_hard_retracel(self.current_phase, tick)
            self.ps.append(self.current_phase)
            ticks = self.current_phase.get_last_nsec_ticks(Settings.CONFIRM_TICKS)
            self.current_phase = ContinuationPhase(ticks=ticks, pm=self)
        except ph.PhaseTerminatedException:
            self.on_hard_retracel(self.current_phase, tick)
            self.on_termination(self.current_phase, tick)
            self.aps.append(self.current_phase)
            ticks = self.current_phase.get_last_nsec_ticks(Settings.CONFIRM_TICKS)
            self.current_phase = ContinuationPhase(ticks=ticks, pm=self)
        except ph.PhaseStartFailed:
            # logger.info(f"phase start failed at: {tick}, phase: {self.current_phase}")
            ticks = self.current_phase.get_last_nsec_ticks(Settings.CONFIRM_TICKS)
            self.current_phase = ContinuationPhase(ticks=ticks, pm=self)
        # self.process_ps(tick)

    def process_ps(self, tick):
        # Process for process stack
        mark_for_removal = []
        for i in range(len(self.ps)):
            phase = self.ps[i]
            try:
                phase.process(tick)
            except ph.PhaseSoftRetraceException:
                # logger.info(f"soft retracel of {phase}")
                pass
            except ph.PhaseHardRetraceException:
                pass
            except ph.PhaseTerminatedException:
                mark_for_removal.append(i)
                self.aps.append(phase)
        mark_for_removal.reverse()
        for mark in mark_for_removal:
            del self.ps[mark]

    def on_phase_start(self, phase, tick):
        self.current_order = ph.Order(type=ph.Order.TYPE_BUY, limit_price=tick.last_price, quantity=Settings.QUANTITY)

    def on_hard_retracel(self, phase, tick):
        if phase.direction == ph.Direction.UP:
            if self.current_order is not None:
                self.current_order.square_off(tick.last_price)
                self.closed_orders.append(ph.PhaseOrder(phase, self.current_order))
                self.current_order = None
            logger.info(f"Retracel {phase}, Up change %: {phase.hard_retraced_at.last_price - phase.started_at.last_price}")
        else:
            logger.info(f"Retracel {phase}, Down change %: {phase.started_at.last_price - phase.hard_retraced_at.last_price}")

instrument = ph.Instrument(name="NIFTY 22650 CALL 7 Mar 2024")
pm = MyPhaseManager(instrument=instrument, settings=Settings)
# for i in range(100):
for i in range(tdf.shape[0]):
    pm.next(tdf.iloc[i].to_dict())

print(pm.current_phase)
# print(pm.ps)
# print(pm.aps)

2024-03-13 09:39:17,737 - INFO - 2409830248.py:63 - [139867032737024] - 
Retracel Continuation: down, HARD RETRACING from Tick: 51391 8.6, v: 0.0 to Tick: 51410 8.45, v: 5600.0, confirmation:Tick: 51397 8.5, v: 4100.0 terminated at: None, hard retracel: Tick: 51410 8.45, v: 5600.0, soft: None, conf: -62.50000000000006, Down change %: 0.05000000000000071
2024-03-13 09:39:17,742 - INFO - 2409830248.py:61 - [139867032737024] - 
Retracel Continuation: up, HARD RETRACING from Tick: 51405 8.4, v: 17850.0 to Tick: 51428 8.65, v: 8600.0, confirmation:Tick: 51410 8.45, v: 0.0 terminated at: None, hard retracel: Tick: 51428 8.65, v: 8600.0, soft: None, conf: 59.99999999999986, Up change %: 0.20000000000000107
2024-03-13 09:39:17,748 - INFO - 2409830248.py:63 - [139867032737024] - 
Retracel Continuation: down, HARD RETRACING from Tick: 51423 8.85, v: 2550.0 to Tick: 51467 7.9, v: 2600.0, confirmation:Tick: 51429 8.65, v: 18100.0 terminated at: None, hard retracel: Tick: 51467 7.9, v: 2600.0, soft

before u: 0.35000000000000053, d: -0.04999999999999982
after u: 0.35000000000000053, d: -0.15000000000000036, ch: -0.10000000000000053, -1 id: 51469, -2 id: 51467
before u: 0.35000000000000053, d: -0.15000000000000036
after u: 0.35000000000000053, d: -0.20000000000000018, ch: -0.04999999999999982, -1 id: 51469, -2 id: 51469
before u: 0.35000000000000053, d: -0.20000000000000018
after u: 0.35000000000000053, d: -0.25, ch: -0.04999999999999982, -1 id: 51470, -2 id: 51469
before u: 0.35000000000000053, d: -0.25
after u: 0.35000000000000053, d: -0.25, ch: 0.0, -1 id: 51470, -2 id: 51470
before u: 0.35000000000000053, d: -0.25
after u: 0.35000000000000053, d: -0.35000000000000053, ch: -0.10000000000000053, -1 id: 51471, -2 id: 51470
before u: 0.35000000000000053, d: -0.35000000000000053
after u: 0.40000000000000124, d: -0.35000000000000053, ch: 0.05000000000000071, -1 id: 51471, -2 id: 51471
before u: 0.40000000000000124, d: -0.35000000000000053
after u: 0.40000000000000124, d: -0.400000000

2024-03-13 09:39:17,952 - INFO - 2409830248.py:61 - [139867032737024] - 
Retracel Continuation: up, HARD RETRACING from Tick: 51981 8.25, v: 2750.0 to Tick: 51997 8.3, v: 850.0, confirmation:Tick: 51987 8.4, v: 5100.0 terminated at: None, hard retracel: Tick: 51997 8.3, v: 850.0, soft: None, conf: 61.53846153846156, Up change %: -0.09999999999999964
2024-03-13 09:39:17,957 - INFO - 2409830248.py:63 - [139867032737024] - 
Retracel Continuation: down, HARD RETRACING from Tick: 51992 8.45, v: 5800.0 to Tick: 52006 8.35, v: 1550.0, confirmation:Tick: 51998 8.35, v: 2550.0 terminated at: None, hard retracel: Tick: 52006 8.35, v: 1550.0, soft: None, conf: -66.66666666666687, Down change %: 0.0
2024-03-13 09:39:17,960 - INFO - 2409830248.py:61 - [139867032737024] - 
Retracel Continuation: up, HARD RETRACING from Tick: 52003 8.35, v: 2900.0 to Tick: 52010 8.25, v: 300.0, confirmation:Tick: 52008 8.5, v: 0.0 terminated at: None, hard retracel: Tick: 52010 8.25, v: 300.0, soft: None, conf: 60.0,

Continuation: None, INITIATED from Tick: 55794 13.45, v: 16550.0 to Tick: 55799 13.45, v: 3850.0, confirmation:None terminated at: None, hard retracel: None, soft: None, conf: None


In [4]:
pm.closed_orders.sort(key=lambda x: x.order.pnl)

In [5]:
# Losses
# multi_y = [[po.order.limit_price, po.order.square_off_price] for po in pm.closed_orders if po.order.pnl <= 0]
# multi_x = [[po.phase.started_at.id, po.phase.hard_retraced_at.id] for po in pm.closed_orders if po.order.pnl <= 0]
# texts = [str(po.order.pnl) for po in pm.closed_orders if po.order.pnl <= 0]
# subplot = {'xs': multi_x, 'ys': multi_y, 'texts': texts, 'x': [m[1] for m in multi_x], 'y': [m[1] for m in multi_y]}

# All
from bokeh.models import ColumnDataSource

multi_y = [[po.order.limit_price, po.order.square_off_price] for po in pm.closed_orders]
multi_x = [[po.phase.started_at.id, po.phase.hard_retraced_at.id] for po in pm.closed_orders]
texts = [f"{po.order.pnl}, {round(po.phase.confidence_at_confirmation)}, s:{round(po.phase.svac)}, l:{round(po.phase.lvac)}" for po in pm.closed_orders]
subplot = {'xs': multi_x, 'ys': multi_y, 'texts': texts, 'x': [m[1] for m in multi_x], 'y': [m[1] for m in multi_y]}

subplot = ColumnDataSource(subplot)

# phase_in_id = [phase.t_start.id for phase in pm.ps]
# phase_in_price = [phase.t_start.last_price for phase in pm.ps]
# phase_start_plot = ColumnDataSource({'x': phase_in_id, 'y': phase_in_price})

# p_end_id = [phase.t_end.id for phase in pm.ps]
# p_end_price = [phase.t_end.last_price for phase in pm.ps]
# phase_end_plot = ColumnDataSource({'x': p_end_id, 'y': p_end_price})

up_start_id = [phase.t_start.id for phase in pm.ps if phase.direction == ph.Direction.UP]
up_start_price = [phase.t_start.last_price for phase in pm.ps if phase.direction == ph.Direction.UP]
up_start_plot = ColumnDataSource({'x': up_start_id, 'y': up_start_price})

up_end_id = [phase.t_end.id for phase in pm.ps if phase.direction == ph.Direction.UP]
up_end_price = [phase.t_end.last_price for phase in pm.ps if phase.direction == ph.Direction.UP]
up_end_plot = ColumnDataSource({'x': up_end_id, 'y': up_end_price})

up_confirm_id = [phase.started_at.id for phase in pm.ps if phase.direction == ph.Direction.UP]
up_confirm_price = [phase.started_at.last_price for phase in pm.ps if phase.direction == ph.Direction.UP]
up_confirm_plot = ColumnDataSource({'x': up_confirm_id, 'y': up_confirm_price})

down_confirm_id = [phase.started_at.id for phase in pm.ps if phase.direction == ph.Direction.DOWN]
down_confirm_price = [phase.started_at.last_price for phase in pm.ps if phase.direction == ph.Direction.DOWN]
down_confirm_plot = ColumnDataSource({'x': down_confirm_id, 'y': down_confirm_price})

down_start_id = [phase.t_start.id for phase in pm.ps if phase.direction == ph.Direction.DOWN]
down_start_price = [phase.t_start.last_price for phase in pm.ps if phase.direction == ph.Direction.DOWN]
down_start_plot = ColumnDataSource({'x': down_start_id, 'y': down_start_price})

down_end_id = [phase.t_end.id for phase in pm.ps if phase.direction == ph.Direction.DOWN]
down_end_price = [phase.t_end.last_price for phase in pm.ps if phase.direction == ph.Direction.DOWN]
down_end_plot = ColumnDataSource({'x': down_end_id, 'y': down_end_price})

ticks_cdf = ColumnDataSource(pd.DataFrame([tick.__dict__ for tick in pm.ticks]))
# phase_plot.data
import utils as ut
# ut.bokeh_plot(x=list(range(tdf.shape[0])), y=tdf.last_price, x_label="timestamp", y_label="price", plot='line')
# ut.bokeh_plot(x=[m.id for m in pm.ticks], y=[tick.last_price for tick in pm.ticks], x_label="timestamp", y_label="price", plot='line', multi_plots=subplot, subplots=[up_start_plot, up_end_plot, down_start_plot, down_end_plot, up_confirm_plot, down_confirm_plot], subplot_labels=['Up start', 'Up end', 'Down start', 'Down end', 'Up confirm', 'Down confirm'])
ut.bokeh_plot(cds=ticks_cdf, x_label="timestamp", y_label="price", plot='line', multi_plots=subplot, subplots=[up_start_plot, up_end_plot, down_start_plot, down_end_plot, up_confirm_plot, down_confirm_plot], subplot_labels=['Up start', 'Up end', 'Down start', 'Down end', 'Up confirm', 'Down confirm'])
# tdf

Loading BokehJS ...